## LSTM AUTOENCODER

In [14]:
from data_processing import process
filename = 'gear_dataset.csv'
rows = 1000
data = process(filename,rows)
n_rows = data.shape[0]
n_features = data.shape[1]

TypeError: process() takes 1 positional argument but 2 were given

In [6]:
from matplotlib import pyplot
import keras
import tensorflow as tf
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import RepeatVector
from keras.utils import plot_model
from keras.callbacks import EarlyStopping
import numpy as np
from numpy import array


In [11]:
from overlapping_window import overlap_window

time_steps = 40
a = np.r_[0:n_rows]

X_train_samples = overlapping_window(time_steps,20,a)
X_train = data[X_train_samples,:]
X_train = np.squeeze(X_train)
# print(X_train.shape)

X_reversed = np.flip(X_train,1)

In [12]:
n_samples = X_train.shape[0]
time_steps = X_train.shape[1]
n_features = X_train.shape[2]


lstm_initializer = tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5)

encoderLSTM = LSTM(128,return_sequences=True,kernel_initializer =lstm_initializer, recurrent_initializer=lstm_initializer)

# define Encoder
EncoderInputs = Input(shape=(time_steps,n_features))
dense1 =Dense(256, activation='tanh')(EncoderInputs)
dropout = Dropout(0.2)(dense1)
lstm1 = encoderLSTM(dropout)
lstm2, state_h, state_c = LSTM(128,return_sequences=True,return_state=True,kernel_initializer =lstm_initializer, recurrent_initializer=lstm_initializer)(lstm1)
encoder_states = [state_h, state_c]

# define Decoder
lstm3 =  LSTM(128,return_sequences=True,kernel_initializer =lstm_initializer, recurrent_initializer=lstm_initializer)(lstm2,initial_state=encoder_states)
lstm4 = LSTM(128,return_sequences=True,kernel_initializer =lstm_initializer, recurrent_initializer=lstm_initializer)(lstm3)
dense2 = Dense(256, activation='sigmoid')(lstm4)
output = Dense(n_features,activation= 'sigmoid')(dense2)

EncoderDecoder = Model(inputs=EncoderInputs, outputs=output,name="EncoderDecoder")
EncoderDecoder.compile(optimizer='adam', loss='binary_crossentropy')
EncoderDecoder.summary()

Model: "EncoderDecoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 40, 64)]     0                                            
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 40, 256)      16640       input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 40, 256)      0           dense_3[0][0]                    
__________________________________________________________________________________________________
lstm_4 (LSTM)                   (None, 40, 128)      197120      dropout_1[0][0]                  
_____________________________________________________________________________________

In [29]:
def model_builder(hp):
   # do the lstm layers need to have the same number of units? will do that
   # do the two dense layers need to have the same number of units?
    n_samples = X_train.shape[0]
    time_steps = X_train.shape[1]
    n_features = X_train.shape[2]


    lstm_initializer = tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5)
    
    lstm_units = hp.Int('units', min_value=1, max_value=500, step=10)

    # define Encoder
    EncoderInputs = Input(shape=(time_steps,n_features))
    
    dense_units = hp.Int('units', min_value=10, max_value=5000, step=50)
    dense1 =Dense(units=dense_units, activation='tanh')(EncoderInputs)
    
    dropoutP = hp.Float('dropout',min_value=0.0, max_value=0.95, step=0.05)
    dropout = Dropout(dropoutP)(dense1) 

    inputLSTM = dropout
    # Note that all parameter names should be unique (here, in the loop over i, we name the inner parameters 'units_' + str(i))
    for i in range(hp.Int('n_LSTM_layers', 1, 10)):
        #x = layers.LSTM(units=hp.Int('units_' + str(i), min_value=1, max_value=500, step=10), activation='sigmoid')(inputLSTM)
        seq, state_h, state_c = LSTM(units=lstm_units,return_sequences=True,return_state=True,kernel_initializer = lstm_initializer, recurrent_initializer=lstm_initializer)(inputLSTM)
        inputLSTM = seq

    encoder_states = [state_h, state_c]
    
    #decoder
    for i in range(hp.Int('n_LSTM_layers', 1, 10)):
        seq, state_h, state_c = LSTM(units=lstm_units,return_sequences=True,return_state=True,kernel_initializer =lstm_initializer,recurrent_initializer=lstm_initializer,(inputDecoder,initial_state=encoder_states) # tune initializers?
        inputDecoder = seq
        encoder_states = [state_h,state_c]
    
    dense2_units = hp.Int('units', min_value=10, max_value=5000, step=50)
    dense2 = Dense(units=dense2_units, activation='sigmoid')(lstm4)
    
    output = Dense(n_features,activation= 'sigmoid')(dense2)

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    EncoderDecoder = Model(inputs=EncoderInputs, outputs=output,name="EncoderDecoder")
    EncoderDecoder.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), loss='binary_crossentropy', metrics=[tf.keras.metrics.BinaryCrossentropy()])
    #EncoderDecoder.summary()
  

    return EncoderDecoder

SyntaxError: invalid syntax (<ipython-input-29-04d7b4ffa987>, line 33)

In [27]:
import kerastuner as kt
tuner = kt.Hyperband(model_builder,
                     objective='val_loss',
                     max_epochs=10,
                     factor=3,
                     project_name='LSTM')

es= EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10)
tuner.search(X_train, X_train, epochs=50, validation_split=0.2, callbacks=[es])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

INFO:tensorflow:Reloading Oracle from existing project .\LSTM\oracle.json
Traceback (most recent call last):
  File "C:\Users\ellsan\Anaconda3\envs\tf\lib\site-packages\kerastuner\engine\hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-26-9dd9ba830787>", line 43, in model_builder
    recurrent_initializer=lstm_initializer)(inputDecoder,initial_state=encoder_states) # tune initializers?
UnboundLocalError: local variable 'inputDecoder' referenced before assignment
Invalid model 0/5
Traceback (most recent call last):
  File "C:\Users\ellsan\Anaconda3\envs\tf\lib\site-packages\kerastuner\engine\hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-26-9dd9ba830787>", line 43, in model_builder
    recurrent_initializer=lstm_initializer)(inputDecoder,initial_state=encoder_states) # tune initializers?
UnboundLocalError: local variable 'inputDecoder' referenced before assignment
Invalid model 1/5
Traceback 

RuntimeError: Too many failed attempts to build model.

In [23]:
n_samples = X_train.shape[0]
time_steps = X_train.shape[1]
n_features = X_train.shape[2]
import time
train_size = int(np.floor(0.7*n_samples))

model = EncoderDecoder
es= EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10)

s=time.time()

history = model.fit(X_train[0:train_size,:,:], X_train[0:train_size,:,:], validation_data=(X_train[train_size:,:,:], X_train[train_size:,:,:]), epochs=300, verbose=2, shuffle=False, callbacks = [es])

e=time.time()

# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='validation')

pyplot.legend()
pyplot.show()
print(f'training time = {e-s} seconds')

NameError: name 'EncoderDecoder' is not defined